In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException

In [2]:
driver = webdriver.Chrome(executable_path='../chromedriver')

In [3]:
driver.get('https://www.latamairlines.com/ar/es/ofertas-vuelos?dataFlight=%7B%22tripTypeSelected%22%3A%7B%22label%22%3A%22Ida+y+Vuelta%22%2C%22value%22%3A%22RT%22%7D%2C%22cabinSelected%22%3A%7B%22label%22%3A%22Economy%22%2C%22value%22%3A%22Economy%22%7D%2C%22passengerSelected%22%3A%7B%22adultQuantity%22%3A1%2C%22childrenQuantity%22%3A0%2C%22infantQuantity%22%3A0%7D%2C%22originSelected%22%3A%7B%22id%22%3A%22EZE_AR_AIRPORT%22%2C%22name%22%3A%22Ezeiza+Intl.%22%2C%22city%22%3A%22Buenos+Aires%22%2C%22cityIsoCode%22%3A%22BUE%22%2C%22country%22%3A%22Argentina%22%2C%22iata%22%3A%22EZE%22%2C%22latitude%22%3A-34.8222%2C%22longitude%22%3A-58.5358%2C%22timezone%22%3A-3%2C%22tz%22%3A%22America%2FBuenos_Aires%22%2C%22type%22%3A%22AIRPORT%22%2C%22countryAlpha2%22%3A%22AR%22%2C%22allAirportsText%22%3Anull%2C%22airportIataCode%22%3A%22EZE%22%7D%2C%22destinationSelected%22%3A%7B%22id%22%3A%22MAD_ES_AIRPORT%22%2C%22name%22%3A%22Barajas+Intl.%22%2C%22city%22%3A%22Madrid%22%2C%22cityIsoCode%22%3A%22MAD%22%2C%22country%22%3A%22Espa%C3%B1a%22%2C%22iata%22%3A%22MAD%22%2C%22latitude%22%3A40.471926%2C%22longitude%22%3A-3.56264%2C%22timezone%22%3A1%2C%22tz%22%3A%22Europe%2FMadrid%22%2C%22type%22%3A%22AIRPORT%22%2C%22countryAlpha2%22%3A%22ES%22%2C%22allAirportsText%22%3Anull%2C%22airportIataCode%22%3A%22MAD%22%7D%2C%22dateGoSelected%22%3A%222022-01-01T15%3A00%3A00.000Z%22%2C%22dateReturnSelected%22%3A%222022-01-31T15%3A00%3A00.000Z%22%2C%22redemption%22%3Afalse%7D')
try:
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//li[@class="sc-eAudoH rtrCi"]')))
    print('La página se cargó correctamente')
except TimeoutException:
    print('La página tardo demasiado en responder')
    

La página se cargó correctamente


In [4]:
def extraer_vuelos():
    vuelos = driver.find_elements_by_xpath('//li[@class="sc-eAudoH rtrCi"]')
    return vuelos

In [5]:
def extraer_informacion(vuelo):
    horarios = {}
    horario_ida = vuelo.find_elements_by_xpath('.//span[@class="sc-gMcBNU gwIRwm"]')[0].text
    horario_vuelta = vuelo.find_elements_by_xpath('.//span[@class="sc-gMcBNU gwIRwm"]')[1].text
    duración_vuelo =  vuelo.find_element_by_xpath('.//div[@class="sc-izvnbC eqFECQ flight-duration"]//span[@class="sc-iLVFha jrpeUd"]').text
    cantidad_escalas = vuelo.find_element_by_xpath('.//div[@class="sc-blIhvV cUAiPA"]//span').text
    horarios['ida'] = horario_ida
    horarios['vuelta'] = horario_vuelta
    horarios['duracion'] = duración_vuelo
    horarios['escalas'] = cantidad_escalas
    vuelo.find_element_by_xpath('.//div[@class="sc-fdQOMr ibkvFX"]').click()
    time.sleep(0.5)
    tarifas = vuelo.find_elements_by_xpath('.//div[@class="sc-ihiiSJ eSWOit"]')
    lista_tarifas = []
    for tarifa in tarifas:
        lista_tarifas.append(tarifa.find_element_by_xpath('.//span[@class="sc-ijnzTp hUOrir displayAmount"]').text)
    for tarifa in lista_tarifas:
        horarios[f'tarifa_{lista_tarifas.index(tarifa)+1}'] = tarifa
    vuelo.find_element_by_xpath('.//button[@class="MuiButtonBase-root MuiButton-root MuiButton-text xp-Button-null MuiButton-textSizeSmall MuiButton-sizeSmall MuiButton-disableElevation"]').click()
    return horarios

In [6]:
vuelos = extraer_vuelos()
print(f'Se encontraron {len(vuelos)} vuelos')
vuelos_lista_info = []
for vuelo in vuelos:
    vuelos_lista_info.append(extraer_informacion(vuelo))
    print(f'Vuelo {vuelos.index(vuelo) + 1} extraido ({((vuelos.index(vuelo)+1)/len(vuelos))*100}%)')

Se encontraron 8 vuelos
Vuelo 1 extraido (12.5%)
Vuelo 2 extraido (25.0%)
Vuelo 3 extraido (37.5%)
Vuelo 4 extraido (50.0%)
Vuelo 5 extraido (62.5%)
Vuelo 6 extraido (75.0%)
Vuelo 7 extraido (87.5%)
Vuelo 8 extraido (100.0%)


In [7]:
df = pd.DataFrame(vuelos_lista_info)
df

,ida,vuelta,duracion,escalas,tarifa_1,tarifa_2,tarifa_3,tarifa_4
0,7:45,15:25\n+1,27 h 40 min,1 parada*,"ARS 273.689,54","ARS 475.622,96",NaN,NaN
1,11:27,15:50\n+1,24 h 23 min,1 parada*,"ARS 421.129,12","ARS 487.229,26","ARS 549.598,67","ARS 319.592,90"
2,19:06,15:50\n+1,16 h 44 min,1 parada*,"ARS 421.129,12","ARS 487.229,26","ARS 549.598,67","ARS 319.592,90"
3,11:27,13:20\n+1,21 h 53 min,2 paradas*,"ARS 421.129,12","ARS 487.229,26","ARS 549.598,67",NaN
4,11:27,15:25\n+1,23 h 58 min,2 paradas*,"ARS 441.876,30","ARS 511.223,34","ARS 576.646,36","ARS 335.070,04"
5,18:40,15:50\n+2,41 h 10 min,2 paradas*,"ARS 352.274,62","ARS 410.792,43","ARS 467.214,83","ARS 821.949,41"
6,19:06,14:20\n+2,39 h 14 min,2 paradas*,"ARS 559.144,72","ARS 630.811,42","ARS 756.768,31",NaN
7,11:27,23:00\n+1,31 h 33 min,3 paradas*,"ARS 485.060,68","ARS 561.016,42","ARS 632.683,12","ARS 758.640,01"


In [8]:
df.to_excel('LATAM_flights_info.xlsx')